In [ ]:
import loadDataForSKtime

loaderHR = loadDataForSKtime.PatientTimeSeriesLoader(
    "C:/Users/emily/Documents/DissertationProject/training/training_setA_csv", ['HR'])

In [ ]:
VARforecaster = VARmodel.VARForecaster(train_data_sub, test_data_sub, ["HR", "O2Sat"])

In [ ]:
VARforecaster.fit()
VARforecasts = VARforecaster.predict(steps=6)

In [ ]:
VARforecaster.evaluate_model(VARforecasts)

In [ ]:
VARforecaster.plot_forecast(VARforecasts)